In [3]:
import pandas as pd
pd.set_option("display.max_columns",80)
from time import sleep

from nba_api.stats.static import players
from nba_api.stats.endpoints import playerawards, playercareerstats

In [4]:
#get list of all inactive players; to be used for training/testing
inactives=pd.DataFrame(players.get_inactive_players())
inactives


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4568,1627790,Ante Zizic,Ante,Zizic,False
4569,78647,Jim Zoet,Jim,Zoet,False
4570,78648,Bill Zopf,Bill,Zopf,False
4571,78650,Matt Zunic,Matt,Zunic,False


In [5]:
test_df=inactives.iloc[:5].copy()
test_df

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [6]:
# This function creates the label variable - whether or not a player is in the HOF
def get_hof(row):
    # sleeping to respect NBA's rate limiting
    sleep(0.5)
    awards = playerawards.PlayerAwards(row["id"]).get_data_frames()[0]
    return "Hall of Fame Inductee" in awards["DESCRIPTION"].values
test_df["HOF"] = test_df.apply(get_hof, axis=1)
test_df

,id,full_name,first_name,last_name,is_active,HOF
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False,True
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False,False


In [6]:
#function that will be passed to pd rename function to rename average columns
def rename_avgs(col):
    if col=="PF":
        rename="F"
    elif col=="OREB" or col =="DREB":
        rename=col[:2]
    elif col[0]=="F":
        rename=col
    else:
        rename=col[0]
    return rename+"PG"
def clean_avgs(df):
    df.rename(columns=rename_avgs,inplace=True)
    return pd.concat([df.iloc[0,5:8],df.iloc[0,9:11],df.iloc[0,12:14],df.iloc[0,15:]])

In [7]:
def get_career_stats(row):
    # sleep to respect rate limiting
    sleep(0.5)
    #holds the various dfs-season, career, reg season, playoffs
    totals=playercareerstats.PlayerCareerStats(row["id"]).get_data_frames()
    sleep(0.5)
    avgs=playercareerstats.PlayerCareerStats(row["id"],per_mode36="PerGame").get_data_frames()
    #concatenate career regular season and average regular season
    return pd.concat([totals[1].iloc[0,3:],clean_avgs(avgs[1]),totals[3].iloc[0,3:].add_prefix("PF_"),clean_avgs(avgs[3]).add_prefix("PF_")])
#playercareerstats.PlayerCareerStats(76003).get_data_frames()[0].columns

In [7]:
ai=inactives[inactives["id"]==947]
ai

,id,full_name,first_name,last_name,is_active
1928,947,Allen Iverson,Allen,Iverson,False


In [9]:
get_career_stats(ai)

GP          914
GS          901
MIN       37577
FGM        8467
FGA       19906
          ...  
PF_SPG      2.1
PF_BPG      0.2
PF_TPG      3.1
PF_FPG      2.2
PF_PPG     29.7
Name: 0, Length: 74, dtype: object

In [12]:
pd.concat([test_df,test_df.apply(get_career_stats,axis=1)],axis=1)

,id,full_name,first_name,last_name,is_active,HOF,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,MPG,FGMPG,FGAPG,FG3MPG,FG3APG,FTMPG,FTAPG,ORPG,DRPG,RPG,APG,SPG,BPG,TPG,FPG,PPG,PF_GP,PF_GS,PF_MIN,PF_FGM,PF_FGA,PF_FG_PCT,PF_FG3M,PF_FG3A,PF_FG3_PCT,PF_FTM,PF_FTA,PF_FT_PCT,PF_OREB,PF_DREB,PF_REB,PF_AST,PF_STL,PF_BLK,PF_TOV,PF_PF,PF_PTS,PF_MPG,PF_FGMPG,PF_FGAPG,PF_FG3MPG,PF_FG3APG,PF_FTMPG,PF_FTAPG,PF_ORPG,PF_DRPG,PF_RPG,PF_APG,PF_SPG,PF_BPG,PF_TPG,PF_FPG,PF_PPG
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False,False,256,53.0,3200,620,1236,0.501618,0.0,3.0,0.000000,225,321,0.700935,283,563,846,85,71,69,247,484,1465,12.5,2.4,4.8,0.0,0.0,0.9,1.3,1.1,2.2,3.3,0.3,0.3,0.3,1.0,1.9,5.7,17,4.0,106,18,40,0.450000,0.0,0.0,0.000000,2,4,0.500000,3,17,20,3,0,1,11.0,11,38,6.2,1.1,2.4,0.0,0.0,0.1,0.2,0.2,1.0,1.2,0.2,0.0,0.1,0.6,0.6,2.2
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False,False,505,NaN,11023,1769,4138,0.427501,NaN,NaN,NaN,1019,1400,0.727857,503,1124,4065,601,131,205,11,1120,4557,21.8,3.5,8.2,NaN,NaN,2.0,2.8,2.4,5.4,8.0,1.2,0.6,1.0,0.7,2.2,9.0,18,NaN,210,37,70,0.528571,NaN,NaN,NaN,18,26,0.692308,11,27,64,9,1,8,NaN,12,92,11.7,2.1,3.9,NaN,NaN,1.0,1.4,1.0,2.5,3.6,0.5,0.1,0.7,NaN,0.7,5.1
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False,True,1560,1476.0,57446,15837,28307,0.559473,1.0,18.0,NaN,6712,9304,0.721410,2975,9394,17440,5660,1160,3189,2527,4657,38387,36.8,10.2,18.1,0.0,NaN,4.3,6.0,2.4,7.6,11.2,3.6,0.9,2.6,2.7,3.0,24.6,237,140.0,8612,2356,4422,0.532791,0.0,4.0,NaN,1050,1419,0.739958,505,1273,2481,767,189,476,447.0,797,5762,36.3,9.9,18.7,0.0,NaN,4.4,6.0,2.6,6.5,10.5,3.2,1.0,2.4,2.6,3.4,24.3
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False,False,586,336.0,15627,3514,7943,0.442402,474.0,1339.0,0.353996,1051,1161,0.905254,219,868,1087,2079,487,46,963,1106,8553,26.7,6.0,13.6,0.8,2.3,1.8,2.0,0.4,1.5,1.9,3.5,0.8,0.1,1.6,1.9,14.6,15,14.0,415,69,187,0.368984,14.0,49.0,0.285714,43,45,0.955556,5,18,23,35,7,1,22.0,37,195,27.7,4.6,12.5,0.9,3.3,2.9,3.0,0.3,1.2,1.5,2.3,0.5,0.1,1.5,2.5,13.0
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False,False,236,145.0,4808,720,1726,0.417149,18.0,76.0,0.236842,372,529,0.703214,286,490,776,266,184,83,309,485,1830,20.4,3.1,7.3,0.1,0.3,1.6,2.2,1.2,2.1,3.3,1.1,0.8,0.4,1.3,2.1,7.8,13,5.0,178,24,63,0.380952,0.0,3.0,0.000000,20,24,0.833333,17,24,41,11,4,4,4.0,14,68,13.7,1.8,4.8,0.0,0.2,1.5,1.8,1.3,1.8,3.2,0.8,0.3,0.3,0.3,1.1,5.2


In [12]:
#will eventually be combined with get_hof function as both make same api call
#for now, though, need to isolate the logic to get awards
def get_awards(row):
    awards=playerawards.PlayerAwards(row["id"]).get_data_frames()[0]
    return awards.groupby("DESCRIPTION").size()

In [8]:
mj=inactives[inactives["full_name"]=="Michael Jordan"]
mj

,id,full_name,first_name,last_name,is_active
2139,893,Michael Jordan,Michael,Jordan,False


In [10]:
mj_awards=get_awards(mj)

In [13]:
get_awards(mj)

DESCRIPTION
All-Defensive Team                                     9
All-NBA                                               11
All-Rookie Team                                        1
Hall of Fame Inductee                                  1
IBM Award                                              2
NBA All-Star                                          14
NBA All-Star Most Valuable Player                      3
NBA Champion                                           6
NBA Defensive Player of the Year                       1
NBA Finals Most Valuable Player                        6
NBA Most Valuable Player                               5
NBA Player of the Month                               16
NBA Player of the Week                                25
NBA Rookie of the Month                                3
NBA Rookie of the Year                                 1
NBA Sporting News Most Valuable Player of the Year     7
NBA Sporting News Rookie of the Year                   1
Olympic Gold Medal 